***Méthodes numériques: Module 4***

# Grey Scott model

Nous commencons par importer la librairie numpy, matplotlib et matplotlib.cm qui nous permettra de dessiner des **cartes**.
Ensuite nous définissons tous nos paramètres et nous importons le fichier contenant les données initiales u et v qui se trouve dans le fichier data du même dépôt.

In [4]:
import numpy
from matplotlib import pyplot
import matplotlib.cm as cm
%matplotlib inline

In [16]:
n = 192
Du, Dv, F, k = 0.00016, 0.00008, 0.035, 0.065 # Bacteria 1 
dh = 5/(n-1)
T = 8000
dt = .9 * dh**2 / (4*max(Du,Dv))
nt = int(T/dt)
print (nt)

8301


In [63]:
uvinitial = numpy.load('./data/uvinitial.npz')
U = uvinitial['U']
V = uvinitial['V']
print(U,V)

[[ 1.03341932  1.00909902  1.03523907 ...,  1.01236     1.02769296
   1.01185515]
 [ 1.02049366  1.03874603  1.03186003 ...,  1.00917594  1.00648275
   1.01929723]
 [ 1.0297385   1.02874589  1.02488765 ...,  1.0345737   1.00195443
   1.03189674]
 ..., 
 [ 1.03851006  1.01833226  1.00260628 ...,  1.00514165  1.04292736
   1.0090857 ]
 [ 1.03951143  1.0425582   1.03609776 ...,  1.00643229  1.04094253
   1.03869138]
 [ 1.01132752  1.00554757  1.00241105 ...,  1.04435044  1.02683595
   1.01239924]] [[ 0.04286057  0.01747253  0.03235064 ...,  0.0283667   0.03892816
   0.0255146 ]
 [ 0.03567483  0.04898278  0.03607639 ...,  0.02100673  0.0305137
   0.00374435]
 [ 0.04936444  0.0423903   0.00414055 ...,  0.02084643  0.03015715
   0.03085144]
 ..., 
 [ 0.03707045  0.03856759  0.01737007 ...,  0.00421018  0.03256909
   0.00247397]
 [ 0.04781775  0.00190273  0.02729943 ...,  0.02507722  0.0468819
   0.00760259]
 [ 0.01798466  0.02495585  0.04354743 ...,  0.00184759  0.01039884
   0.01832535]]


Le système d'équations différentielles à résoudre s'écrit:


\begin{align}
\frac{\partial u}{\partial t} &= D_u \nabla ^2 u - uv^2 + F(1-u)\\
\frac{\partial v}{\partial t} &= D_v \nabla ^2 v + uv^2 - (F + k)v
\end{align}

## Méthode explicite

Nous allons le discrétiser vers l'avant dans le temps et par un schéma centré dans l'espace (forward-time,centred-space). Nous allons le résoudre par un schéma explicite: la discrétisation dans l'espace s'effectue au temps n:


\begin{align}
\frac{u_{i,j}^{n+1}-u_{i,j}^{n}}{\Delta t}= D_{u}*(\frac{u_{i+1,j}^{n}-2u_{i,j}^{n}+u_{i-1,j}^{n}}{\Delta x^2}+\frac{u_{i,j+1}^{n}-2u_{i,j}^{n}+u_{i,j-1}^{n}}{\Delta y^2})-u_{i,j}^{n}*(v_{i,j}^{n})^2+F(1-u_{i,j}^{n})\\
\frac{v_{i,j}^{n+1}-v_{i,j}^{n}}{\Delta t}=D_{v}*(\frac{v_{i+1,j}^{n}-2v_{i,j}^{n}+v_{i-1,j}^{n}}{\Delta x^2}+\frac{v_{i,j+1}^{n}-2v_{i,j}^{n}+v_{i,j-1}^{n}}{\Delta y^2})+u_{i,j}^{n}*(v_{i,j}^{n})^2-F(1+k)v_{i,j}^{n}
\end{align}

On pose ensuite 
\begin{align}
\Delta x^2=\Delta y^2=\delta^2\\
\Delta t= \frac{9}{40}\frac{\delta^2}{\max(D_u, D_v)}
\end{align}

On met tous les termes connus dans le membre de droite et les termes inconnus dans le membre de gauche:

\begin{align}
u_{i,j}^{n+1}=u_{i,j}^{n}+\Delta t(Du*(\frac{u_{i+1,j}^{n}-2u_{i,j}^{n}+u_{i-1,j}^{n}}{\delta^2}+\frac{u_{i,j+1}^{n}-2u_{i,j}^{n}+u_{i,j-1}^{n}}{\delta^2})-u_{i,j}^{n}*(v_{i,j}^{n})^2+F(1-u_{i,j}^{n}))\\
v_{i,j}^{n+1}=v_{i,j}^{n}+\Delta t(Dv*(\frac{v_{i+1,j}^{n}-2v_{i,j}^{n}+v_{i-1,j}^{n}}{\delta^2}+\frac{v_{i,j+1}^{n}-2v_{i,j}^{n}+v_{i,j-1}^{n}}{\delta^2})+u_{i,j}^{n}*(v_{i,j}^{n})^2-F(1+k)v_{i,j}^{n})
\end{align}
c'est-à-dire:
\begin{align}
u_{i,j}^{n+1}=u_{i,j}^{n}*(1-4\frac{\Delta t*Du}{\delta^2})+\Delta t(Du*(\frac{u_{i+1,j}^{n}+u_{i-1,j}^{n}+u_{i,j+1}^{n}+u_{i,j-1}^{n}}{\delta^2})-u_{i,j}^{n}*(v_{i,j}^{n})^2+F(1-u_{i,j}^{n}))\\
v_{i,j}^{n+1}=v_{i,j}^{n}*(1-4\frac{\Delta t*Dv}{\delta^2})+\Delta t(Dv*(\frac{v_{i+1,j}^{n}+v_{i-1,j}^{n}+v_{i,j+1}^{n}+v_{i,j-1}^{n}}{\delta^2})+u_{i,j}^{n}*(v_{i,j}^{n})^2-F(1+k)v_{i,j}^{n})
\end{align}

### Paramètres:

In [129]:
n = 192
Du, Dv, F, k = 0.00016, 0.00008, 0.035, 0.065 # Bacteria 1 
dh = 5/(n-1) #delta
T = 8000
dt = .9 * dh**2 / (4*max(Du,Dv))
nt = int(T/dt)
print(k)

0.065


In [134]:
def greyscott(u0,v0):
    ucopy=u0.copy()
    vcopy=v0.copy()
    for k in range (80):
        uk=ucopy.copy()
        vk=vcopy.copy()
        ucopy[1:-1,1:-1]=uk[1:-1,1:-1]*(1-4*dt*Du/dh**2)+dt*Du*(uk[2:,1:-1]+uk[:-2,1:-1]+uk[1:-1,2:]+uk[1:-1,:-2])/(dh**2)\
                        -dt*uk[1:-1,1:-1]*(vk[1:-1,1:-1])**2+dt*F*(1-uk[1:-1,1:-1])
        vcopy[1:-1,1:-1]=vk[1:-1,1:-1]*(1-4*dt*Dv/dh**2)+dt*Dv*(vk[2:,1:-1]+vk[:-2,1:-1]+vk[1:-1,2:]+vk[1:-1,:-2])/(dh**2)\
                        +dt*uk[1:-1,1:-1]*vk[1:-1,1:-1]**2-dt*F*(1+k)*vk[1:-1,1:-1]
        #conditions de Neumann:
        ucopy[:,0]=ucopy[:,1]
        ucopy[0,:]=ucopy[1,:]
        ucopy[:,191]=ucopy[:,190]
        ucopy[191,:]=ucopy[190,:]
        vcopy[:,0]=vcopy[:,1]
        vcopy[0,:]=vcopy[1,:]
        vcopy[:,191]=vcopy[:,190]
        vcopy[191,:]=vcopy[190,:]
    return (ucopy,vcopy)

In [135]:
sol=greyscott(U,V)
print(sol[0][100])

[  1.00119866e+00   1.00119866e+00   1.00088296e+00   1.00033630e+00
   9.99741569e-01   9.99175872e-01   9.98864904e-01   9.98686398e-01
   9.98822041e-01   9.99034075e-01   9.99446450e-01   9.99858545e-01
   1.00032547e+00   1.00072747e+00   1.00105511e+00   1.00125173e+00
   1.00130042e+00   1.00120019e+00   1.00096798e+00   1.00063677e+00
   1.00020748e+00   9.99733916e-01   9.99155390e-01   9.98604011e-01
   9.97928861e-01   9.97280322e-01   9.96063009e-01   9.93687866e-01
   9.86232016e-01   9.65876027e-01   8.91622063e-01   7.44653716e-01
   1.91156296e-01   7.63266593e-03  -1.72910214e+00   1.03300323e-01
  -2.17392971e+00   7.05980521e-01  -2.16576486e+00   1.01726274e-01
   4.44784145e-01  -1.90868232e+00   1.73510913e+01  -2.30546306e+01
  -3.20634594e+01  -1.42404173e+02   1.28983979e+02  -1.34375794e+04
   1.67749934e+01  -1.17098539e+05   3.86776098e+03  -1.08357704e+05
   3.94904360e+03  -1.31483149e+05   2.15419823e+03  -3.90384542e+04
   1.14860694e+02  -6.07313045e+00

## Méthode implicite

Cette fois-ci nous utilisons la méthode implicite: la discrétisation dans l'espace s'effectue au temps n+1:
\begin{align}
\frac{u_{i,j}^{n+1}-u_{i,j}^{n}}{\Delta t}= D_{u}*(\frac{u_{i+1,j}^{n+1}-2u_{i,j}^{n+1}+u_{i-1,j}^{n+1}}{\Delta x^2}+\frac{u_{i,j+1}^{n+1}-2u_{i,j}^{n+1}+u_{i,j-1}^{n+1}}{\Delta y^2})-u_{i,j}^{n}*(v_{i,j}^{n})^2+F(1-u_{i,j}^{n})\\
\frac{v_{i,j}^{n+1}-v_{i,j}^{n}}{\Delta t}=D_{v}*(\frac{v_{i+1,j}^{n+1}-2v_{i,j}^{n+1}+v_{i-1,j}^{n+1}}{\Delta x^2}+\frac{v_{i,j+1}^{n+1}-2v_{i,j}^{n+1}+v_{i,j-1}^{n+1}}{\Delta y^2})+u_{i,j}^{n}*(v_{i,j}^{n})^2-F(1+k)v_{i,j}^{n})
\end{align}
